In [ ]:
# default_exp get_ip

# get_ip

> A toolset for .

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# |export
import subprocess
import os
import yaml
import re

def get_key(dict,value):
    
    for k, v in dict.items():
        for v,v1 in v.items():
            for v1,v2 in v1.items():
                if v2 == value:
                    return k,v
                
def myip():

    ret_byte = subprocess.check_output(['ifconfig'])
    ret_str = ret_byte.decode('utf-8')
    # Cut string from 'equal symbol' to 'degree C symbol', then convert to float
    en = ret_str[ret_str.find('eno1:'): ret_str.find('lo')]
    ip = en[en.find('inet')+5: en.find('netmask')-2]
    return ip

def whoami(data):
    ip = myip(data)
    machine,device = get_key(data,ip)
    return machine,device
    
def get_xbee_address(dict,value):
    for k, v in dict.items():
        for v,v1 in v.items():
            for v1,v2 in v1.items():
                if v2 == value:
                    address = dict[k]["rpi"]['xbee_rx']
                    return address
                
def get_xbee_address_boat(dict,value):
    pair_device = dict[value]["xbee"]["xbee_pair"]
    address = dict[pair_device]["rpi_2"]["xbee_rx"]
    return address

def find_duckiepond_devices_yaml(yaml_filename="duckiepond-devices.yaml"):
    dp_yaml_path = ""
    for root, dirs, files in os.walk(os.path.expanduser('~')):
        for name in files:
            if name == yaml_filename:
                dp_yaml_path = os.path.abspath(os.path.join(root, name))
                break
    return dp_yaml_path

def dp_load_config(dp_yaml_path):
    dp_dict = {}
    with open(dp_yaml_path, 'r') as stream:
        try:
            dp_dict = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return dp_dict

def dp_get_devices(dp_yaml_path, pattern='boat*'):
    dp_dict = dp_load_config(dp_yaml_path)
    devices = []
    for key in dp_dict.keys():
        match = re.match(pattern, key)
        if match:
            devices.append(key)
    return devices

def device_get_sensors(dict, device='sensor1'):
    sensors = []
    for key in dp_dict[device]['topics'].keys():
        sensors.append(key)
    return sensors

def sensor_get_topic(dp_dict, device='sensor1', find='zed'):
    topic = dp_dict[device]['topics'][find]
    return topic

#ssh functions will not give testing example for now on, since hostname will depend on your running machine

def ssh_ping_nano(hostname):
    response = os.system("ssh $USER@" + hostname + " ping -c 1 192.168.0.100")
    return response

def ssh_ping_rpi(hostname):
    response = os.system("ssh $USER@" + hostname + " ping -c 1 192.168.0.101")
    return response

def test_ssh_intranet():
    error = []
    for ip in hostnames:
        num = ssh_ping_rpi(ip)
        if num!=0:
            error.append("ssh ping rpi error " + ip)
        num = ssh_ping_nano(ip)
        if num!=0:
            error.append("ssh ping nano error " + ip)
    assert not error, "errors occured:\n{}".format("\n".join(error))
    
def ssh_connection(hostname):
    response = os.system("ssh $USER@" + hostname + " date")
    return response

def test_ssh():
    error = []
    for ip in hostnames:
        num = ssh_connection(ip)
        if num != 0:
            error.append("ssh error " + ip)
    assert not error, "errors occured:\n{}".format("\n".join(error))

def ip_connection(hostname):
    response = os.system("ping -c 1 " + hostname)
    return response

def test_ping():
    error = []
    for ip in hostnames:
        num = ip_connection(ip)
        if num != 0:
            error.append("Network Error " + ip)
    assert not error, "errors occured:\n{}".format("\n".join(error))

def ssh_rostopic(hostname, rosversion="melodic"):
    response = os.system('ssh $USER@' + hostname + ' "source /opt/ros/"' + rosversion + '"/setup.bash && rostopic list"')
    return response

def test_rostopic():
    error = []
    for ip in hostnames:
        num = ssh_rostopic(ip)
        if num != 0:
            error.append("ssh rostopic list error " + ip)
    assert not error, "errors occured:\n{}".format("\n".join(error))


## Testing example

In [ ]:
cd data

[Errno 2] No such file or directory: 'data'
/home/arg/arg_robotics_tools/data


In [ ]:
ip = myip()
print(ip)

140.113.148.77


### To use this library, dp_yaml_path and dp_load_config is most important functions, do it first

In [ ]:
dp_yaml_path = find_duckiepond_devices_yaml("duckiepond-devices-machine.yaml")
print(dp_yaml_path)

/home/arg/arg_robotics_tools/data/duckiepond-devices-machine.yaml


In [ ]:
dp_dict = dp_load_config(dp_yaml_path)
print(dp_dict)

{'basestation': {'ip': '192.168.1.77'}, 'boat1': {'xbee': {'xbee_pair': 'anchor1'}, 'rpi': {'ip': '192.168.1.10', 'ip_in': '192.168.0.100', 'xbee_rx': '41AF1BAD', 'hostname': 'dtrpiboat01'}, 'nano': {'ip': '192.168.1.13', 'ip_in': '192.168.0.106', 'xbee_tx': '41AF1B1A', 'hostname': 'dtnanoboat01'}}, 'boat2': {'xbee': {'xbee_pair': 'anchor2'}, 'rpi': {'ip': '192.168.1.20', 'ip_in': '192.168.0.100', 'xbee_rx': '41AF1B4B', 'hostname': 'dtrpiboat02'}, 'nano': {'ip': '192.168.1.23', 'ip_in': '192.168.0.106', 'xbee_tx': '41AF1AD4', 'hostname': 'dtnanoboat02'}}, 'boat3': {'xbee': {'xbee_pair': 'anchor3'}, 'rpi': {'ip': '192.168.1.30', 'ip_in': '192.168.0.100', 'xbee_rx': '417E5F98', 'hostname': 'dtrpiboat03'}, 'nano': {'ip': '192.168.1.33', 'ip_in': '192.168.0.106', 'xbee_tx': '417E629D', 'hostname': 'dtnanoboat03'}}, 'boat4': {'xbee': {'xbee_pair': 'anchor4'}, 'rpi': {'ip': '192.168.1.40', 'ip_in': '192.168.0.100', 'xbee_rx': '41AF1C55', 'hostname': 'dtrpiboat04'}, 'nano': {'ip': '192.168.1.

Below code will catch your local ip so the device you run this code should be in the yaml file, if you use your laptop to run it, eventually get the error, no mind!

In [ ]:
machine,device = whoami(dp_dict)
xbee_address = get_xbee_address_boat(dp_dict,machine)

print(machine)
print(device)
print(xbee_address)

### dp_get_devices is the function to list out what device we have in yaml, look carefull below of how to use

In [ ]:
boats = dp_get_devices(dp_yaml_path, 'boat*')
print("load boats: {}".format(boats))

load boats: ['boat1', 'boat2', 'boat3', 'boat4', 'boat5', 'boat6', 'boat7', 'boat8']


In [ ]:
dtkvbots = dp_get_devices(dp_yaml_path, 'dt-kv*')
print("load bots: {}".format(dtkvbots))

load bots: ['dt-kv-01', 'dt-kv-02', 'dt-kv-03', 'dt-kv-04', 'dt-kv-05', 'dt-kv-06', 'dt-kv-07']


In [ ]:
anchors = dp_get_devices(dp_yaml_path, 'anchor*')
print("load anchors: {}".format(anchors))

load anchors: ['anchor1', 'anchor2', 'anchor3', 'anchor4', 'anchor5', 'anchor6', 'anchor7', 'anchor8']


In [ ]:
sensors = dp_get_devices(dp_yaml_path, 'sensor*')
print("load sensors: {}".format(sensors))

load sensors: ['sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6']


### device_get_sensors and sensor_get_topic could use together as below, it will be very convenienc even work with dp_get_devices

In [ ]:
sensors = device_get_sensors(dp_dict, 'sensor4')
print(sensors)

['dreamvu', 'mmwave0', 'mmwave1', 'mmwave2', 'mmwave3']


In [ ]:
for sensor in sensors:
    topic = sensor_get_topic(dp_dict, 'sensor4', sensor)
    print(topic)

dreamvu/pal/get/left
radar_0/ti_mmwave/ti_mmwave/radar_scan_pcl
radar_1/ti_mmwave/ti_mmwave/radar_scan_pcl
radar_2/ti_mmwave/ti_mmwave/radar_scan_pcl
radar_3/ti_mmwave/ti_mmwave/radar_scan_pcl
